# Setup Collaboration
In this notebook, we establish a collaboration between an ArcGIS Online Organization and an ArcGIS Enterprise system, with the Online Organization as the host. There will be a group added to the collaboration in Enterprise, and another in AGOL. After the notebook has run, these groups will share data and items by copy (if possible) and by reference (if copy is not possible). Server Service items are the only things know that cannot be synced by copy and will have to be synced by reference. After the notebook has run, there will be a folder in each org's content that will have the same title as the collaboration title and will contain all the content from the other participant in the collaboration.


## Getting Started:
1. Ensure that there is a group set up that you will use for collaboration in both Online and Enterprise. These can be empty or have items in them and do not need to have the same name.
2. Find the group IDs for the collaboration groups in ArcGIS Online and Enterprise. These can be found by navigating to the group page and looking in the URL (for example, if the URL is http://envisioning.maps.arcgis.com/home/group.html?id=a7903db4086641b98570bce5856a6364#overview, the group ID is "a7903db4086641b98570bce5856a6364".
3. Ensure that "Enable Sync" is checked on each item's setting page in each group.
4. Change the variables in cell 1. The Host and Guest accounts need to be admins. Example variables:
    - HOST_URL = "https://envisioning.maps.arcgis.com"
    - HOST_USERNAME = "portaladmin" 
    - GUEST_URL = "https://pdoscriptdev.esri.com/portal"
    - GUEST_USERNAME = "admin"
    - HOST_GROUP_ID = "4d7ff4f81d6340428ef290b7de801204"
    - COLLAB_NAME = "Enterprise and Online"
    - WORKSPACE_NAME = "Sample Workspace"
    - DESCRIPTION = "Data sharing sample between Enterprise and Online"
    - EXPIRATION = 24
    - LOCAL_TMP_DIR = "/Users/joesmith/Documents/tmp" (Mac) or "C:/Users/joesmith/Documents/tmp" (Windows)
5. Run the notebook cell by cell.
    - In code cell 3, you will need to enter the passwords for the Host and Guest organizations when prompted.
    - Code cells 4 and 10 will show the guest and host groups to be added to the collaboration - ensure they are correct.
    - Code cell 13 will print the collaborations in the host and guest GIS. They should both show the COLLAB_NAME you specified.
6. After running in order to make the content sync sooner than 24 hours, you need to open up the Enterprise Portal, go to Organization > Collaborations > click on the collaboration name > click on the gear icon under Action, and then click Edit Workspace. Click sync at scheduled intervals and set it to repeat as frequently as you'd like, so it will do the first sync whenever you specify. Make sure this dialog box also says Feature layers are sent as copies, not references. The workspace will say that sync failed, even if it has not tried to sync yet. After the first sync that you schedule manually has happened, you can check if the content sync has succeeded or failed. These steps will be automated in a future version of this script.

## Troubleshooting: 
After the first scheduled sync, you can view the logs on the portal machine (if the portal url is "https://pdoscriptdev.esri.com/portal", go to "https://pdoscriptdev.esri.com/portal/portaladmin/logs/") to see if there were any errors with the sync (ex. layers that were not able to sync by copy). If any layers failed to sync by copy and are synced by reference instead, they will need to be unshared and reshared, after the issue is fixed. You can find more information about this in the [`Collaboration FAQs`](https://enterprise.arcgis.com/en/portal/latest/administer/windows/common-questions-for-distributed-collaboration.htm). If a feature layer syncs by copy, the feature layer will show as feature layer (hosted) for the guest, but if it syncs by reference, it will just show as a feature layer.


In [ ]:
# Global Variables Set by User **change these values before running script**

# host organization is the ArcGIS Online Org - needs to be admin account
HOST_URL = "https://ps-cc.maps.arcgis.com"
HOST_USERNAME = "gbushong_PS_CC"

# guest organization is the Enterprise Portal - needs to be admin account
GUEST_URL = "https://amicaent.esri.com/portal"
GUEST_USERNAME = "admin"

# this group in the host org contains items to share in the collaboration or is empty
# this should be empty most of the time
HOST_GROUP_ID = "6fd586038d76406594d72ed2cb348e41" #aec-test-group

# this group in the guest org contains items to share in the collaboration or is empty
GUEST_GROUP_ID = "c14c069f2b9a4f72905c49e28373394d" #aec-enterprise-group

# this will be the name of the collaboration and the name of the folder created in enterprise
COLLAB_NAME = "Enterprise + Online"

# name of the workspace that the groups will be added to
WORKSPACE_NAME = "Enterprise Workspace"

# description for the collaboration and workspace
DESCRIPTION = "Data sharing sample between Enterprise and Online"

# host contact info
HOST_FIRST = "Joe"
HOST_LAST = "Smith"
HOST_EMAIL = "jsmith@sample.com"

# guest contact info
GUEST_FIRST = "Judy"
GUEST_LAST = "Smith"
GUEST_EMAIL = "jsmith1@sample.com"

# local folder to temporarily save invitation and response docs into
LOCAL_TMP_DIR = "/Users/grac9792/OneDrive - Esri/pdo-scripts"

In [ ]:
# import libraries
from arcgis.gis import GIS
from arcgis.gis import Group
from arcgis.gis.admin import Collaboration

In [ ]:
# connect to host GIS, the new ArcGIS Online instance
print("Connecting to {} with username {}...".format(HOST_URL, HOST_USERNAME))
host_gis = GIS(HOST_URL, HOST_USERNAME)

# connect to guest GIS, the ArcGIS Enterprise instance
print("Connecting to {} with username {}...".format(GUEST_URL, GUEST_USERNAME))
guest_gis = GIS(GUEST_URL, GUEST_USERNAME, verify_cert=False)

In [ ]:
# search for sharing groups in host gis
host_group = Group(host_gis, HOST_GROUP_ID)
host_group

In [ ]:
# create a collaboration
host_collab = host_gis.admin.collaborations.create(name=COLLAB_NAME, description=DESCRIPTION,
                                                  workspace_name=WORKSPACE_NAME,
                                                  workspace_description=DESCRIPTION,
                                                  portal_group_id=host_group.id,
                                                  host_contact_first_name=HOST_FIRST,
                                                  host_contact_last_name=HOST_LAST,
                                                  host_contact_email_address=HOST_EMAIL,
                                                  access_mode='sendAndReceive')

collab_id = host_collab['id']
host_collab

In [ ]:
# set privileges for the new workspace in the new collaboration
config = [{host_collab.workspaces[0]['id']:'sendAndReceive'}]

# invite the guest GIS to the collaboration and download the invitation file
invite_file = host_collab.invite_participant(config_json=config, expiration=24, 
                                             guest_portal_url=GUEST_URL,
                                             save_path=LOCAL_TMP_DIR)

# print the path to the invite file
invite_file

In [ ]:
# accept the invitation at the participant GIS
guest_gis.admin.collaborations.accept_invitation(first_name=GUEST_FIRST, last_name=GUEST_LAST,
                                                 email=GUEST_EMAIL,
                                                 invitation_file=invite_file)

In [ ]:
# get the correct guest collaboration based off shared collaboration id
guest_collab = Collaboration(guest_gis.admin.collaborations, collab_id)
type(guest_collab)

In [ ]:
# now export a response file that can be sent back to the host GIS
response_file = guest_collab.export_invitation(out_folder=LOCAL_TMP_DIR)
response_file

# accept the collaboration response at the host GIS
host_collab.import_invitation_response(response_file)

In [ ]:
# get guest portal id
guest_portal_id = guest_collab.workspaces[0]['participantsWithoutGroupLinks'][0]['portalId']
print(guest_portal_id)

host_portal_id = guest_collab.workspaces[0]['participantGroupLinks'][0]['portalId']
print(host_portal_id)

# find the guest groups to add to collaboration workspace
guest_group = Group(guest_gis, GUEST_GROUP_ID)
guest_group

In [ ]:
# join a group from the guest GIS to the collaboration
guest_collab.add_group_to_workspace(portal_group=guest_group, workspace=guest_collab.workspaces[0])

# get workspace ID
workspace_id = guest_collab.workspaces[0]['id']

# get portal ID
host_portal_id = guest_collab['collaborationHostPortalId']
host_portal_id

In [ ]:
# verify the collaboration
print("Collaborations on host GIS:")
for collab in host_gis.admin.collaborations.list():
    print(collab.name)
    
print("-----------------------------")
print("Collaborations on guest GIS:")
for collab in guest_gis.admin.collaborations.list():
    print(collab.name)